In [275]:
import pandas as pd
import math
import datetime
from datetime import datetime
from datetime import date
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
import numpy
import pickle


In [276]:
mat_training_0 = loadtxt("Train1.txt", delimiter="  ", unpack=False)
mat_training_1 = loadtxt("Train2.txt", delimiter="  ", unpack=False)
mat_testing_0 = loadtxt("Test1.txt", delimiter="  ", unpack=False)
mat_testing_1 = loadtxt("Test2.txt", delimiter="  ", unpack=False)

#only has two features and can add more by concatenation and is to test a example model
X_train = np.concatenate((mat_training_0,mat_training_1), axis=0)

print(X_train[0])
X_test = np.concatenate((mat_testing_0,mat_testing_1), axis=0)

[-0.587726  5.10592 ]


In [277]:
labels_0_tr = np.array([1]*mat_training_0.shape[0])
labels_1_tr = np.array([2]*mat_training_1.shape[0])
labels_0_te = np.array([1]*mat_testing_0.shape[0])
labels_1_te = np.array([2]*mat_testing_1.shape[0])

#the testing classes are grouped under one vector
y_test = np.concatenate((labels_0_te,labels_1_te), axis=0)
y_train = np.concatenate((labels_0_tr,labels_1_tr), axis=0)

In [278]:
#  Data=[list1,list2]
#  X = Data[0]
#  y = Data[1],
#X = [[0, 0], [1, 1]]
#y = [0, 1] 
#y = [0,0,0,0,0]

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.43, random_state=0)
pca = PCA(n_components=2)# adjust yourself
pca.fit(X_train)
pca.fit(X_test)


X_t_train = pca.transform(X_train)
X_t_test = pca.transform(X_test)

clf = SVC()
clf.fit(X_t_train, y_train)
print('score', clf.score(X_t_test, y_test))
#print('pred label', clf.predict(X_t_test))

with open ('model_pickle', 'wb') as filename:
   pickle.dump(clf,filename)

score 0.829


In [279]:
with open ('model_pickle', 'rb') as filename:
   modelRetrieved = pickle.load(filename)

In [280]:
modelRetrieved.fit(X_t_train, y_train)
print('score', modelRetrieved.score(X_t_test, y_test))

score 0.829


In [281]:
#model testing above works to 82% need to be tuned for new data with greater amount of columns
#pca is performed and a sample model is trained and the model is saved as variable then retrieved

In [ ]:
#creating the matrix with labels below

In [267]:
filename1 = pd.read_excel('CGMData670GPatient3.xlsx')
filename2 = pd.read_excel('InsulinAndMealIntake670GPatient3.xlsx')

In [137]:
#print(filename2.Index[0])
CGMData_pre = []
InsulinData_pre = []

temp = filename1['Sensor Glucose (mg/dL)'].to_numpy()
Eats =  filename2['BWZ Carb Input (grams)'].to_numpy()
InsulinDate = filename2['Date'].to_numpy()
InsulinTime = filename2['Time'].to_numpy()

h = filename2['Time'].to_numpy()
p = filename2['Date'].to_numpy()


# CGMData = filename1[['Date','Time','Sensor Glucose (mg/dL)']]
# InsulinData = InsulinData['Date'],InsulinData['Time'],InsulinData['Alarm']

In [138]:
regular = []
for i in Eats:
    regular.append(i)

In [139]:
#print(type(regular))
#print(regular[0])
newarray = regular[::-1]

In [140]:
#print(newarray[0])
#print(regular[-1])

In [141]:
for i in Eats:
    A = []
     #print(float(i))

In [142]:
#print(temp[0])
#print(h[1000])
#print(int(str(h[1000]).split(':',3)[0]),"int")
InsulinTimeAfter = []
#print(int(str(InsulinTime[1000]).split(':',3)[0]), "insulin time")
for i in InsulinTime:
    arr = []
    arr.append(int(str(i).split(':',3)[0]))
    arr.append(int(str(i).split(':',3)[1]))
    arr.append(int(str(i).split(':',3)[2]))
    InsulinTimeAfter.append(arr)

#all times for insulin data
InsulinTimeAfterReversed = InsulinTimeAfter[::-1]

#for i in InsulinTimeAfterReversed:
#    print(i)
#print(str(h[1000]).split(':',3)[1])
#print(str(h[1000]).split(':',3)[2])

#print(type(p[1]),"here")
InsulinDateAfter = []
for i in InsulinDate:
    arr = []
    var = pd.to_datetime(i)
    q= str(var).split()
    arr.append(int(q[0].split('-',3)[0]))
    arr.append(int(q[0].split('-',3)[1]))
    arr.append(int(q[0].split('-',3)[2]))
    InsulinDateAfter.append(arr)

DateandTime = []
for i in range(0, len(InsulinDateAfter),1):
    arr = []
    arr.append(InsulinDateAfter[i])
    arr.append(InsulinTimeAfter[i])
    DateandTime.append(arr)
    
#print(DateandTime[0])
#print(DateandTime[-1])
#print(len(DateandTime))

#dates and time linked and reversed from start to end

DateAndFood = []
for i in range(0, len(DateandTime),1):
    arr = []
    arr.append(DateandTime[i])
    arr.append(Eats[i])
    DateAndFood.append(arr)
    
DateAndFood = DateAndFood[::-1]
print(DateAndFood[0])

[[[2017, 9, 5], [8, 25, 33]], nan]


In [120]:
DataClean = []
for i in DateAndFood:
    arr = []
    empty = math.isnan(i[1])
    if i[1] > 0 and empty == False:
        arr.append(i[0])
        arr.append(i[1])  
        DataClean.append(arr)
        

In [200]:
def separateDate(Data,Data1):
    
    noMeal = False
    diff_in_hours = 0
        
    lowerYear = Data[0][0]
    lowerMonth = Data[0][1]
    lowerDay = Data[0][2]
    
        
    lowerHour = Data[1][0]
    lowerMinute = Data[1][1]
    lowerSecond = Data[1][2]
    
    upperYear = Data1[0][0]
    upperMonth = Data1[0][1]
    upperDay = Data1[0][2]
        
    upperHour = Data1[1][0]
    upperMinute = Data1[1][1]
    upperSecond = Data1[1][2]
        
    date_1 = str(lowerDay) + '/'+ str(lowerMonth) +'/'+ str(lowerYear) + ' ' +str(lowerHour)+':'+ str(lowerMinute)+':'+ str(lowerSecond)
    date_2 = str(upperDay) + '/'+ str(upperMonth) +'/'+ str(upperYear) + ' ' +str(upperHour)+':'+ str(upperMinute)+':'+ str(upperSecond)
    date_format_str = '%d/%m/%Y %H:%M:%S'
        
    start = datetime.strptime(date_1, date_format_str)
    end =   datetime.strptime(date_2, date_format_str)
    # Get interval between two timstamps as timedelta object
    diff = end - start
    # Get interval between two timstamps in hours
    diff_in_hours = diff.total_seconds() / 3600
        
    if diff_in_hours >= 2:
        noMeal = True
        
    return noMeal

In [203]:
#print(DataClean[0])
#print(DataClean[1])

for i in range(0, len(DataClean), 1):
    
    if i < len(DataClean):
        print(separateDate(DataClean[i][0],DataClean[i+1][0]))


True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
False
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
Tru

IndexError: list index out of range

In [ ]:
#print(var)

In [ ]:
q= str(var).split()

In [ ]:
#print(q[0].split('-',3)[0])
#print(q[0].split('-',3)[1])
#print(q[0].split('-',3)[2])

In [78]:
# for col in range(0, len(CGMData[0:-1]),1):
#     if col < len(CGMData[0:-1]):
#         #print(CGMData[col:col+1])      
#         row = []
#         row.append(CGMData[col:col+1]['Date'])
#         CGMData_pre.append(row)

# #         row.append(CGMData['Date'](col))
# #         row.append(CGMData['Time'](col))
#          #row.append(CGMData['Sensor Glucose (mg/dL)'](col))
 

# # for col .thisin InsulinData:
# #     row = []
# #     row.append(col['Date'])
# #     row.append(col['Time'])
# #     row.append(col['Alarm'])
# #     InsulinData_pre.append(row)  

In [79]:
# print(CGMData_pre[0])